# Regression of Vote Characteristics on CAR

In [65]:
import stata_setup

stata_setup.config('/Applications/StataNow', 'mp')

In [66]:
%%stata

clear all
set more off
version 18

* Paths
global PROCESSED_DATA "processed_data"
global TABLES "tables"
global EVENT_WINDOW 5

In [ ]:
%%stata
foreach stage in created end {
    import delimited using "$PROCESSED_DATA/event_study_panel_`stage'.csv", clear

    * Parse date
    gen day = date(date, "YMD")
    format day %td
    gen year = year(day)

    * Keep CAR at the end of event window
    keep if index==$EVENT_WINDOW

    * Encode categorical variables
    encode gecko_id, gen(token)
    encode space, gen(dao)

    * Encode vote characteristics
    local vars half_vp_ratio vn_hhi vs_hhi cci

    eststo clear
    foreach v of local vars {
        reghdfe car `v', absorb(dao year) vce(cluster dao)
        eststo `v'
    }

    * optional table
    esttab half_vp_ratio vn_hhi vs_hhi cci, se star(* 0.10 ** 0.05 *** 0.01)

    * save results
    esttab using "$TABLES/reg_car_votes_`stage'.tex", replace se star(* 0.10 ** 0.05 *** 0.01) ///
        label mtitles("Half VP Ratio" "$HHI^{\text{vn}}$" "$HHI^{\text{vs}}$" "$CCI$")
}

SystemError: 
----------------------------------------------------------------------------
                      (1)             (2)             (3)             (4)   
                      car             car             car             car   
----------------------------------------------------------------------------
half_vp_ra~o       0.0276                                                   
                  (0.113)                                                   

vn_hhi                             0.0607                                   
                                 (0.0450)                                   

vs_hhi                                             0.0833*                  
                                                 (0.0439)                   

cci                                                                0.0407   
                                                                 (0.0645)   

_cons             -0.0152         -0.0614*        -0.0831*